In [ ]:
!pip install folium

In [ ]:
!pip install geopy

CARTO BIO CO2

In [10]:
import folium
from folium import plugins
import pandas as pd
from geopy.geocoders import Nominatim # pour les coordonnées sat à partir d'une adresse postale

In [7]:
# import des données / création du df
df_rebours = pd.read_excel('./data/bio_co2.xlsx', sheet_name='Rebours_')
df_injections = pd.read_excel('./data/bio_co2.xlsx', sheet_name='Injections_')
df_zibacs =  pd.read_excel('./data/bio_co2.xlsx', sheet_name='Zibacs_')
df_fermentations = pd.read_excel('./data/bio_co2.xlsx', sheet_name='Fermentations_')
df_industriels = pd.read_excel('./data/bio_co2.xlsx', sheet_name='Industriels_')

# preprocessing
df_injections = df_injections.iloc[:615] ## on garde que ces lignes là pour supprimer les lignes vides de l'excel
df_fermentations = df_fermentations.dropna()

df_fermentations.head()
#print(df_injections[df_injections['Lng'].isna()].index)
#df_injections['Lng'].dropna()
#df_injections['Lat'].dropna()
#type_ = df_injections.dtypes
#print (type_)
#print(len(df_injections))

,Ville,Société,CO2_capté,Lat,Lng
0,Artenay,Tereos,150.0,48.0833,1.8833
1,Villette/Aube,Cristal Union,140.0,48.5333,4.1000
2,Connantre,Tereos,110.0,48.7167,3.9167
3,Marckolsheim,Tereos,140.0,48.1647,7.5440
4,Beinheim,Roquette,150.0,48.8667,8.0833


In [41]:
# Adjuster la position le départ et le niveau de zoom initial
# définition de la position -> Juste donner les lattitude et longitude dans un tuple(Nord, Est) -> attention si vos coordonnées sont 74°W il faut les mettre en -74°E
start_position = (48.866667 , 2.33333) #coordonnées de Paris
start_position = (46.7167 , 2.5167) #coordonnées de Saint Amand
# Création d'une variable dans laquelle on crée une map de base avec un style graphique au choix tiles = Stamen (Terrain, toner, Watercolor) ou tiles = CartoDB(positron, dark_matter )
base_map = folium.Map(tiles ='CartoDB dark_matter', location = start_position, zoom_start= 7 )


# ajout d'un calque de contour/frontière
#layer1 = folium.GeoJson()

# ajout d'un marker
folium.Marker(location = start_position,
              tooltip='',
              icon = folium.Icon(icon ='road', prefix='fa', color='red'), # aller sur glyphicons getbootstraps par exemple pour récupérer les noms
              popup= folium.Popup('<p>toutes les infos</p>', max_width=500) 
              )#.add_to(base_map)

# logo GRT ou Designlab
from folium.plugins import FloatImage
#url = 'https://upload.wikimedia.org/wikipedia/fr/thumb/5/57/LogoGRTgaz2023.svg/1200px-LogoGRTgaz2023.svg.png'
logo = folium.CustomIcon(icon_image='./ressources/logo_designlab_proto.png', icon_size=(100, 100))
FloatImage('./ressources/logo_designlab_proto.png', bottom=2, left=1, height = '28px', width = '260px').add_to(base_map)



# ajout d'un gestionnaire de calque
# definition du style de bordure
borderStyleReg={
    "color":"grey",
    "weight" : 2,
    "fill" : False 
}

borderStyleDep={
    "color":"blue",
    "weight" : 2,
    "fill" : False 
}

# chargement des frontière des régions à partir d'un geojson
folium.GeoJson(data = './data/regions.geojson',
               name="Régions",
               style_function =lambda x: borderStyleReg,
               )#.add_to(base_map)


# chargement des frontière des départements à partir d'un geojson
folium.GeoJson(data = './data/departements.geojson',
               name="Départements",
               style_function =lambda x: borderStyleDep,
               )#.add_to(base_map)


# FONCTION CALQUE CIRCLE
def create_circle_layer(map, data, layer_name, lat_col, lng_col, debit_col, color_circle):
    marker_layer = folium.FeatureGroup(layer_name).add_to(map)

    for i in range(len(data)):
        latVal = data.iloc[i][lat_col]
        lngVal = data.iloc[i][lng_col]
        #typologieVal = data.iloc[i][type_col]
        debitVal = data.iloc[i][debit_col]
        colorVal = color_circle #if typologieVal == 'Rebours' else "red"
        cRad = debitVal * 100

        folium.Circle(
            location=[latVal, lngVal],
            radius=cRad,
            color=colorVal,
            weight=2,
            fill=True
        ).add_to(marker_layer)


# Création des calques
create_circle_layer(base_map, df_rebours, 'Rebours', 'Lat', 'Lng', 'DébitCO2_ktparan', 'blue')
create_circle_layer(base_map, df_injections, 'Injections', 'Lat', 'Lng', 'Production_CO2_ktparan', 'red')
create_circle_layer(base_map, df_fermentations, 'Fermentations', 'Lat', 'Lng', 'CO2_capté', 'green')
create_circle_layer(base_map, df_industriels, 'Industriels', 'Lat', 'Lng', 'Quantité_CO2_ktparan', 'orange')



# CALQUE ZIBACS
zibacsLayer = folium.FeatureGroup('ZIBACS').add_to(base_map)

for j in range(len(df_zibacs)):
    latVal = df_zibacs.iloc[j]['Lat']
    lngVal = df_zibacs.iloc[j]['Lng']
    name = df_zibacs.iloc[j]['Ville']
    popupContent = folium.Popup(" Test contenu de fenêtre popup",
                                max_width = 500)
    #typologieVal = df_zibacs.iloc[j]['Type de site']
    #colorVal = "red"

    folium.Marker(
        location =[latVal, lngVal],
        tooltip = name,
        popup = popupContent,
        icon = folium.Icon(icon='star', prefix='glyphicon', color = 'red'),
        
    ).add_to(zibacsLayer)



# MENU CALQUES
# Créez la boîte de contrôle des couches avec tous les paramètres
layer_control = folium.LayerControl(
    position='topleft',  # Position en haut à gauche
    collapsed=False,     # Boîte de contrôle réduite par défaut
    autoZIndex=True,    # Ajuste automatiquement les z-index des couches
    collapsed_title='Couches',  # Titre de la boîte de contrôle réduite
    overlay=True,        # Gérer les couches overlay
    base=False, # Gérer les couches de base
    title = 'TEST' 
)






# Ajoutez la boîte de contrôle à la carte
layer_control.add_to(base_map)

#menu simple
#folium.LayerControl().add_to(base_map)

# affiche dans jupiter
base_map

In [ ]:
# Création d'une variable dans laquelle on crée une map de base avec un style graphique au choix tiles = Stamen (Terrain, toner, Watercolor) ou tiles = CartoDB(positron, dark_matter )
base_map = folium.Map(tiles ='CartoDB dark_matter')

# base map est un folium map object pour vérifier on check avec type, qui contient la map entière
type(base_map)
base_map

In [9]:
# sauver une map là où est l'executable
base_map.save('index.html')

# on peut vérifier en regardant avec la commande
!ls

# on peut l'ouvrir (va l'ouvrir dans le navigateur)
!open index.html

CO2_map.ipynb data          index.html    ressources
